In [1]:
import re
import requests
import pandas as pd

In [ ]:
ensembl_annotation = 

In [87]:
missing = pd.read_excel('../reports/missing_hek_select_20220518.xlsx')

In [2]:
peptides = pd.read_csv('missing_select_peptides_20220518.txt', sep='\t')

In [120]:
peptides['entry'] = peptides['entry'].map(lambda x: x + '-1' if '-' not in x else x)

In [20]:
peptides = peptides.drop_duplicates(['peptide'])

In [121]:
missing[missing['uniprot_isoform'].str.contains('Q3B7I2-1')]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [14]:
peptides['gene'] = peptides['entry'].map(missing.drop_duplicates('uniprot_isoform').set_index('uniprot_isoform')['ensembl_gene_name'])

In [17]:
peptides['entries'] = peptides.groupby('peptide')['entry'].transform(lambda x: ', '.join(x))

In [122]:
len(peptides[peptides['entry'] == 'Q3B7I2-1'])

9

In [3]:
def get_gpmdb_hits(peptide):
    pat = re.compile('\[(.*?)\]')
    res = requests.get(f'http://rest.thegpm.org/1/peptide/count/seq={peptide}').text
    
    try:
        return int(re.search(pat, res).group(1))
    except AttributeError:
        return np.nan

In [5]:
requests.get(f'http://rest.thegpm.org/1/peptide/count/seq=SDLCETSANHTEL').text

'[12]'

In [117]:
#peptides['gpmdb_count'] = peptides.peptide.map(get_gpmdb_hits)

In [139]:
x = requests.get('https://api.nextprot.org/entries/search/peptide?peptide=EIQSTLRER&no-variant-match=true').text

In [140]:
import json

In [141]:
y = json.loads(x)

In [142]:
y

[{'uniqueName': 'NX_Q2VPJ9',
  'overview': {'history': {'proteinExistenceUniprot': {'name': 'transcript level',
     'description': 'Evidence at transcript level',
     'descriptionName': 'Evidence_at_transcript_level',
     'level': 2},
    'nextprotIntegrationDate': 1267570588727,
    'nextprotUpdateDate': 1644986687234,
    'uniprotIntegrationDate': 1162288800000,
    'uniprotUpdateDate': 1632909600000,
    'uniprotVersion': '110',
    'lastSequenceUpdate': 1136847600000,
    'sequenceVersion': '1',
    'formattedNextprotUpdateDate': '2022-02-16',
    'formattedUniprotIntegrationDate': '2006-10-31',
    'formattedUniprotUpdateDate': '2021-09-29',
    'formattedNextprotIntegrationDate': '2010-03-02'},
   'families': [{'familyId': 117046,
     'accession': 'FA-05261',
     'name': 'LRRC75',
     'level': 'Family',
     'description': 'Belongs to the LRRC75 family.',
     'region': None,
     'parent': None}],
   'proteinNames': [{'clazz': 'PROTEIN_NAMES',
     'type': 'name',
     'qu

In [143]:
for i in y[0]['annotationsByCategory']['pepx-virtual-annotation']:
    for j in i['targetingIsoformsMap']:
        print(j)

NX_Q2VPJ9-1
NX_Q2VPJ9-2


In [116]:
#y[0]

In [147]:
def check_nextprot_unique(peptide):
    matches = []
    res = requests.get(f'https://api.nextprot.org/entries/search/peptide?peptide={peptide}&no-variant-match=true').text
    
    res = json.loads(res)
    for i in res:
        for j in i['annotationsByCategory']['pepx-virtual-annotation']:
            for l in j['targetingIsoformsMap']:
                matches.append(l[3:])
            
    if len(matches) == 1:
        return str(matches[0])
    else:
        matches.sort()
        return ', '.join(matches)

In [148]:
check_nextprot_unique('EIQSTLRER')

'Q2VPJ9-1, Q2VPJ9-2'

In [84]:
def get_nextprot_peptides(isoform):
    sequences = []
    miscleavages = []
    
    res = requests.get(f'https://api.nextprot.org/digestion/NX_{isoform}?protease=TRYPSIN&minpeplen=7&maxpeplen=25&maxmissedcleavages=1').text
    
    res = json.loads(res)
    
    for i in res:
        sequences.append(i['sequence'])
        miscleavages.append(i['producedWithMiscleavageCount'])
        
    return pd.DataFrame(data={'uniprot_isoform': isoform, 'peptide': sequences, 'miscleavages': miscleavages})

In [73]:
z = json.loads(requests.get(f'https://api.nextprot.org/digestion/NX_Q6IPT4-1?protease=TRYPSIN&minpeplen=7&maxpeplen=25&maxmissedcleavages=1').text)

In [126]:
#get_nextprot_peptides('Q6IPT4-1')

In [83]:
z

[{'sequence': 'TFFVLSQESSSEQLPWSYQEK', 'producedWithMiscleavageCount': 0},
 {'sequence': 'YVESWRVGDTAFWR', 'producedWithMiscleavageCount': 1},
 {'sequence': 'WEAAQASK', 'producedWithMiscleavageCount': 0},
 {'sequence': 'ELVSCCR', 'producedWithMiscleavageCount': 0},
 {'sequence': 'CYQMGLMSR', 'producedWithMiscleavageCount': 0},
 {'sequence': 'AYTPISPANAEGYFEVLIK', 'producedWithMiscleavageCount': 0},
 {'sequence': 'THFGHLGQDLIKELVSCCR', 'producedWithMiscleavageCount': 1},
 {'sequence': 'GPESQSCPSK', 'producedWithMiscleavageCount': 0},
 {'sequence': 'VRFALPGNSQLGLRPGQHLILR', 'producedWithMiscleavageCount': 1},
 {'sequence': 'ELVSCCRR', 'producedWithMiscleavageCount': 1},
 {'sequence': 'CYQMGLMSRYVESWR', 'producedWithMiscleavageCount': 1},
 {'sequence': 'DIARCLLCAGLTEDSYFLF', 'producedWithMiscleavageCount': 1},
 {'sequence': 'RKPFALVCGSAEFTK', 'producedWithMiscleavageCount': 1},
 {'sequence': 'VGDTAFWR', 'producedWithMiscleavageCount': 0},
 {'sequence': 'TFESIYLK', 'producedWithMiscleavage

In [89]:
missing2 = pd.read_excel('../reports/missing_hek_select_20220518.xlsx', index_col=[0,1,2,3]).reset_index()

In [92]:
missing3 = missing2[missing2['uniprot_isoform'].isin(['Q3B7I2-1', 'Q6IPT4-1', 'Q6IPT4-4', 'Q2VPJ9-1', 'Q8NHV5-1', 'Q8N819-3', 'Q9Y6U7-1', 'Q6ZRR5-1', 'Q6ZRR5-3', 'Q6ZRR5-4', 'Q6P7N7-1', 'Q96FV3-2', 'Q96FV3-3', 'Q96FV3-4'])]

In [111]:
isoforms = [
    'Q3B7I2-1', 'Q6IPT4-1', 'Q6IPT4-4', 'Q2VPJ9-1', 'Q8NHV5-1', 
    'Q8N819-3', 'Q9Y6U7-1', 'Q6ZRR5-1', 'Q6ZRR5-3', 'Q6ZRR5-4', 
    'Q6P7N7-1', 'Q96FV3-2', 'Q96FV3-3', 'Q96FV3-4'
]

peptide_df = pd.DataFrame()

for i in isoforms:
    peptide_df = pd.concat([peptide_df, get_nextprot_peptides(i)])

In [112]:
df

,uniprot_isoform,peptide,miscleavages
0,Q3B7I2-1,SDLCETSANHTEL,0
1,Q3B7I2-1,KLYFYSDAYRPLK,1
2,Q3B7I2-1,LYFYSDAYRPLK,0
3,Q3B7I2-1,LEEDPVTK,0
4,Q3B7I2-1,GDKIYQEFK,1
...,...,...,...
17,Q96FV3-4,LELEQQGFIHTK,0
18,Q96FV3-4,HVFFGLGGLYPEPTFKNW,1
19,Q96FV3-4,HQHFQEPEVGCCGK,0
20,Q96FV3-4,LKLELEQQGFIHTK,1


In [123]:
df['gene'] = df['uniprot_isoform'].map(ensembl_annotation.drop_duplicates('uniprot_isoform')
                                       .set_index('uniprot_isoform')['ensembl_gene_name'])

In [124]:
df['entries'] = df.groupby('peptide')['uniprot_isoform'].transform(lambda x: ', '.join(x))

In [131]:
df = df.drop_duplicates('peptide')

In [133]:
df['uniprot_base'] = df['uniprot_isoform'].map(lambda x: x[:x.index('-')])

C:\Users\Dynev\AppData\Local\Temp\ipykernel_12000\3311982611.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['uniprot_base'] = df['uniprot_isoform'].map(lambda x: x[:x.index('-')])


In [134]:
df

,uniprot_isoform,peptide,miscleavages,gene,entries,uniprot_base
0,Q3B7I2-1,SDLCETSANHTEL,0,CNPY1,Q3B7I2-1,Q3B7I2
1,Q3B7I2-1,KLYFYSDAYRPLK,1,CNPY1,Q3B7I2-1,Q3B7I2
2,Q3B7I2-1,LYFYSDAYRPLK,0,CNPY1,Q3B7I2-1,Q3B7I2
3,Q3B7I2-1,LEEDPVTK,0,CNPY1,Q3B7I2-1,Q3B7I2
4,Q3B7I2-1,GDKIYQEFK,1,CNPY1,Q3B7I2-1,Q3B7I2
...,...,...,...,...,...,...
14,Q96FV3-3,GPNDWNLNIYFNCTDLNPSR,0,TSPAN17,"Q96FV3-3, Q96FV3-4",Q96FV3
18,Q96FV3-3,AYRDDIDLQNLIDFAQEYWSCCGAR,1,TSPAN17,"Q96FV3-3, Q96FV3-4",Q96FV3
11,Q96FV3-4,HVFFGLGGLYPEPTFK,0,TSPAN17,Q96FV3-4,Q96FV3
13,Q96FV3-4,AVKANWSK,1,TSPAN17,Q96FV3-4,Q96FV3


In [149]:
df['nextprot_matches'] = df['peptide'].map(check_nextprot_unique)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_12000\1486023350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nextprot_matches'] = df['peptide'].map(check_nextprot_unique)


In [103]:
ensembl_annotation = pd.read_csv('../data/processed/ensembl_annotation_trs_uniprot_20220429.csv', low_memory=False)

In [158]:
def get_multiple_bases(s: str):
    if ', ' in s:
        s = list({i[:i.index('-')] for i in s.split(', ')})
        return ', '.join(sorted(s))
    else:
        return s[:s.index('-')]

In [159]:
df['match_bases'] = df['nextprot_matches'].map(get_multiple_bases)

C:\Users\Dynev\AppData\Local\Temp\ipykernel_12000\1025612744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['match_bases'] = df['nextprot_matches'].map(get_multiple_bases)


In [166]:
df['isoform_unique'] = ~df['nextprot_matches'].str.contains(', ')

C:\Users\Dynev\AppData\Local\Temp\ipykernel_12000\973953685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isoform_unique'] = ~df['nextprot_matches'].str.contains(', ')


In [162]:
df['entry_unique'] = df['uniprot_base'] == df['match_bases']

C:\Users\Dynev\AppData\Local\Temp\ipykernel_12000\3818631663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['entry_unique'] = df['uniprot_base'] == df['match_bases']


In [168]:
df = df.rename({'entries': 'uniprot_isoforms',
                'miscleavages': 'miscleavage_count',
                'nextprot_matches': 'nextprot_match',
                'match_bases': 'nextprot_match_base',
                'isoform_unique': 'unique_for_isoform',
                'entry_unique': 'unique_for_entry'}, axis=1)

In [177]:
df = df.rename({'nextprot_match': 'nextprot_matches'}, axis=1)

In [181]:
get_gpmdb_hits('SDLCETSANHTEL')

nan

In [180]:
df['gpmdb_count'] = df['peptide'].map(get_gpmdb_hits)

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [6]:
import numpy as np

In [7]:
df = pd.read_csv('../data/interim/peptides.tsv', sep='\t')

In [9]:
df.loc[:, 'gpmdb_count'] = df['peptide'].map(get_gpmdb_hits)

In [12]:
df = df.drop(['Unnamed: 0', 'uniprot_isoform', 'uniprot_base', 'nextprot_match_base'], axis=1)

In [14]:
df = df[['gene', 'uniprot_isoforms', 'peptide', 'miscleavage_count', 'gpmdb_count', 'nextprot_matches', 'unique_for_isoform', 'unique_for_entry']]

In [15]:
df = df.rename({'gene': 'ensembl_gene_name'}, axis=1)

In [17]:
df.to_excel('../reports/selection/missing_select_peptides.xlsx', index=False)